# Vents Group - Version 1
OOI Data Labs Workshop - March 2019

A possible interactive would include timeseries of 3D-Temp and Earthquakes that allows students to see the larger context, while viewing video stills from 2 different scenes.

Potetial timeframe: June 2018-January 2019 (or to present)

In [ ]:
# Notebook Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Earthquake Data - First Attempt
Lets try the automatic approach first.

In [ ]:
# Load in Earthquake data
eq_url = 'http://axial.ocean.washington.edu/hypo71.dat'
data = pd.read_fwf(eq_url)
data.head()

This looks pretty good, but we need to reformat several columns.

In [ ]:
# Convert time
data['DateTime'] = pd.to_datetime(data['yyyymmdd'].astype(str) + ' ' + data['HHMMSSS.SS'].astype(str), format='%Y%m%d %H%M %S.%f')

# Convert Lat and Lon
def dm2dd(s):
  degrees, minutes, = s.split()
  dd = float(degrees) + float(minutes)/60
  return dd

data['Latitude'] = data['Lat(D M)'].apply(dm2dd)
data['Longitude'] = -data['Lon(D M)'].apply(dm2dd) #Add negative for West

# Split the MW and NWR columns
new = data['MW NWR'].str.split(' ', n=1, expand=True)
data['MW'] = new[0].astype(float)
data['NWR'] = new[1].astype(float)
data.drop(columns=['yyyymmdd','HHMMSSS.SS','Lat(D M)','Lon(D M)','MW NWR'], inplace=True)

data.head()

In [ ]:
# Quickplot
fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, figsize=(10,6))
data.plot(ax=ax1, x='DateTime',y='Depth',marker='.',linestyle='');
data.plot(ax=ax2, x='DateTime',y='MW',marker='.',linestyle='');

## Earthquake Data - Second Attempt
This time, we'll explicitely extract just the columns we need.  However, we'll still need to do some conversions, so this doesn't save us much code, but it does save a few lines.

In [ ]:
pd.read_fwf?
pd.read

In [ ]:
data = pd.read_fwf(eq_url, colspecs=[(0,19),(20,28),(29,38),(40,45),(47,52)])

# Convert time
data.index = pd.to_datetime(data['yyyymmdd HHMMSSS.SS'].astype(str), format='%Y%m%d %H%M %S.%f')
data.index.name = 'DateTime'

data['Latitude'] = data['Lat(D M)'].apply(dm2dd)
data['Longitude'] = -data['Lon(D M)'].apply(dm2dd) #Add negative for West

data.tail()

## Simple Earthqake map

In [ ]:
# Let's plot the quakes
plt.plot(data.Longitude,data.Latitude,marker='.',linestyle='');
xlim = plt.xlim()
ylim = plt.ylim()

# Add the OOI Sites
sites = pd.read_csv('https://github.com/seagrinch/data-team-python/raw/master/infrastructure/sites.csv')
plt.plot(sites.longitude,sites.latitude,'d',markersize=8)

# Reset the limits
plt.xlim(xlim)
plt.ylim(ylim);

## Hexbin map
First, let's create a hexbin map the regular way, but we'll set the aspect ratio of the plot so it's close to a Mercator projection.

In [ ]:
fig, ax = plt.subplots()
plt.hexbin(data.Longitude,data.Latitude,gridsize=200,bins='log')
plt.colorbar();
plt.plot(sites.longitude,sites.latitude,'d',markersize=8)
plt.xlim(-130.1,-129.9)
plt.ylim(45.8,46.1);
aspect_ratio = np.cos(np.mean(plt.ylim())*np.pi/180)
ax.set_aspect(aspect_ratio);

We can also get a little fancier and also use cartopy so we get the proper map projection.

In [ ]:
!apt-get -qq install python-cartopy python3-cartopy
import cartopy.crs as ccrs

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-130.2,-129.8,45.8,46.1], crs=ccrs.PlateCarree())
plt.hexbin(data.Longitude,data.Latitude,gridsize=200,bins='log',transform=ccrs.Geodetic())
plt.colorbar();
plt.plot(sites.longitude,sites.latitude,'d',markersize=8)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False

## Daily Earthquake Averages

In [ ]:
data_sub = data.loc['2018-6-1':'2019-6-1']

fig,(ax1,ax2) = plt.subplots(2,1,figsize=(8,6),sharex=True,sharey=False)
# data_sub['MW'].resample('1H').mean().plot(ax=ax1,label='Hourly Mean',marker='.',markersize=1,linestyle='');
data_sub['MW'].resample('D').mean().plot(ax=ax1,label='Daily Mean');
data_sub['MW'].resample('D').median().plot(ax=ax1,label='Daily Median');
ax1.legend();

data_sub['MW'].resample('D').count().plot(ax=ax2,label='Count');

ax1.set_ylabel('Magnitude')
ax2.set_ylabel('Count');
ax1.set_title('Daily Earthquakes at Axial');

## TMPSF Data
Next will request and process data from the Thermistor array at Axial Seamount.

### Request Data

In [ ]:
# More setup
import requests
import os
import re
import xarray as xr
! pip install netcdf4==1.5.0

In [ ]:
def request_data(reference_designator,method,stream,start_date,end_date):
  site = reference_designator[:8]
  node = reference_designator[9:14]
  instrument = reference_designator[15:]
  
  # Create the request URL
  api_base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'
  data_request_url ='/'.join((api_base_url,site,node,instrument,method,stream))

  # All of the following are optional, but you should specify a date range
  params = {
    'beginDT':start_date,
    'endDT':end_date,
    'format':'application/netcdf',
    'include_provenance':'true',
    'include_annotations':'true'
  }

  # Make the data request
  r = requests.get(data_request_url, params=params, auth=(API_USERNAME, API_TOKEN))
  data = r.json()
  
  # Return just the THREDDS URL
  return data['allURLs'][0] 

In [ ]:
API_USERNAME = ''
API_TOKEN = ''

The next line is used to request a dataset from the OOI Data Portal.  

**Note, this line only needs to be run once** and then I recommend commenting it out so you don't accidentlally rerun it again. When you run it, simply save the outputted URL in the next line of code, so you can then grab the data.

Of course, if you wish to grab a different time range, including more recent data, you will need to rerun the request line.  Also, note that it will take a few minutes for new data requests to be processed, so you will not be able to contiue on with the notebook until the data files are ready.


In [ ]:
# Data Request
# request_data('RS03ASHS-MJ03B-07-TMPSFA301','streamed','tmpsf_sample',
#              '2018-06-01T00:00:00.000Z','2019-06-01T00:00:00.000Z')


### Load and Process Data

In [ ]:
# This is the output URL from the request_data line
data_url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/sage@marine.rutgers.edu/20190515T024738-RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/catalog.html'

In [ ]:
def get_data(url,bad_inst=''):
  tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
  datasets = requests.get(url).text
  urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
  x = re.findall(r'(ooi/.*?.nc)', datasets)
  for i in x:
    if i.endswith('.nc') == False:
      x.remove(i)
  for i in x:
    try:
      float(i[-4])
    except:
      x.remove(i)
  datasets = [os.path.join(tds_url, i) for i in x]
  
  # Remove extraneous files if necessary
  selected_datasets = []
  for d in datasets:
    if (bad_inst) and bad_inst in d:
      pass
    else:
      selected_datasets.append(d)
#   print(selected_datasets)
  
  # Load in dataset
  ds = xr.open_mfdataset(selected_datasets)
  ds = ds.swap_dims({'obs': 'time'}) # Swap the primary dimension
  ds = ds.chunk({'time': 1000}) # Used for optimization
  ds = ds.sortby('time') # Data from different deployments can overlap so we want to sort all data by time stamp.
  return ds

In [ ]:
# Grab the data
tmpsf_data = get_data(data_url)

Note, for plotting, we will use the 3 center thermistors (1=bottom, 14=mid, and 15=top), as defined in the [TMPSF DPS](https://oceanobservatories.org/instrument-class/tmpsf/).  You can easily plot others as well.

In [ ]:
# Let's make a plot
fig,ax = plt.subplots(4,1,figsize=(8,8),sharex=True)

# And the Earthquake dataset at the top
data.loc['2018-6-1':'2019-6-1']['MW'].resample('D').count().plot(ax=ax[0]);
ax[0].set_ylabel('Eq Count')

# Now add some selected thermistors
thermistors = [1,14,15]
spa = 1 # Subplot axes
for thermistor in thermistors:
  # Due to a quirk, I had to add .to_dataframe() to the next line so both variables are DataFrames
  tmpsf_data["temperature%02d" % (thermistor)].resample(time='1D').mean().to_dataframe().plot(ax=ax[spa],legend=False)
  ax[spa].set_ylabel("temperature%02d" % (thermistor))
  spa = spa+1
  

### Export the dataset
The next few lines will concatenate the earthquake dataset along with a few selected thermistors so we can export the full (daily averaged) dataset as a CSV file for use in Excel or an interactive widget.

In [ ]:
# First will create the averages as xarrays
eq1 = data.loc['2018-6-1':'2019-6-1']['MW'].resample('D').count()
eq1.index.name = 'time'
eq1.name = 'Eq Count'
eq1 = eq1.to_xarray()

eq2 = data.loc['2018-6-1':'2019-6-1']['MW'].resample('D').mean()
eq2.index.name = 'time'
eq2.name = 'Eq Magnitude'
eq2 = eq2.to_xarray()

a = tmpsf_data["temperature01"].resample(time='1D').mean()
b = tmpsf_data["temperature14"].resample(time='1D').mean()
c = tmpsf_data["temperature15"].resample(time='1D').mean()

In [ ]:
# Now we'll merge the datasets and convert to a pandas dataframe
# This could potentially be done the other way around, but I found this order easier.
x = xr.merge([a,b,c,eq1,eq2]).to_dataframe()

# Print the first few rows
x.head()

In [ ]:
# Quickplot
x.plot();

In [ ]:
# Export to CSV
x.to_csv('axial_data.csv')